## IMPORTS

In [2]:
!pip install datasets transformers
# ==4.28.0
!pip install scikit-multilearn
!pip install accelerate -U
!pip install tensorflow
%env SM_FRAMEWORK=tensorflow.keras
!pip install keras
!pip install nltk
!pip install torch
!pip3 install nvidia-tensorrt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 120.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.8 MB/s eta 0:00:00
env: SM_FRAMEWORK=tensorflow.keras
  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-8.6.1-py2.py3-none-any.whl size=16972 sha256=0d2ef4ec62e7466f5a36343e4f

In [3]:
import pickle
import pandas as pd
import numpy as np
import nltk
import torch
import string
import os
import tensorflow as tf
import time
from time import sleep
from string import punctuation
from sklearn import metrics
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import layers, models
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, TFBertModel
from datasets import load_dataset, load_from_disk, DatasetDict, Dataset
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.applications.resnet50 import ResNet50
from tqdm import tqdm
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping
from PIL import Image

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
os.environ['CUDA_VISIBLE_DEVICES'] ='0'

## Load Data

In [ ]:
long_df = pd.read_csv('long_sypnosis.csv').drop(columns = "Unnamed: 0")

In [ ]:
corrupted_list = []

for img in tqdm(range(long_df.shape[0])):
    try:
        images_path = f"/home/jessica/code/chooj202/movie_genre_prediction/notebooks/merged/{long_df['imdb_id'][img]}.jpg"
        image = Image.open(images_path)
    except:
        corrupted_list.append(long_df['imdb_id'][img])

100%|█████████████████████████████████████████████████████████████████████████████| 4624/4624 [00:02<00:00, 1812.31it/s]


In [ ]:
len(corrupted_list)

681

In [ ]:
long_df = long_df.loc[~long_df['imdb_id'].isin(corrupted_list)]

In [ ]:
long_df = long_df.reset_index(drop=True)

## Preprocess Data
images -> resize and scale
text -> word_ids?

In [ ]:
long_df["genre"] = (
    long_df["genre"]
    .apply(eval)
    .apply(lambda x: [genre.strip() for genre in x])
    )

In [ ]:
long_df.head()

,movie,imdb_id,genre,image_url,sypnosis
0,Ramayana: The Legend of Prince Rama,tt0259534,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,An anime adaptation of the Hindu epic the Rama...
1,The Dark Knight,tt0468569,"[Action, Crime, Drama]",http://img.omdbapi.com/?i=tt0468569&h=600&apik...,When the menace known as the Joker wreaks havo...
2,The Lord of the Rings: The Return of the King,tt0167260,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,Gandalf and Aragorn lead the World of Men agai...
3,Spider-Man: Across the Spider-Verse,tt9362722,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt9362722&h=600&apik...,"Miles Morales catapults across the Multiverse,..."
4,The Message,tt0075143,"[Action, Adventure, Biography]",http://img.omdbapi.com/?i=tt0075143&h=600&apik...,"The story of prophet ""Muhammad"" and the delive..."


In [ ]:
def cleaning(sentence):

    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercase
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers

    # Advanced cleaning
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## remove punctuation

    return sentence

In [ ]:
long_df['sypnosis'] = long_df['sypnosis'].apply(cleaning)

In [ ]:
long_df.head()

,movie,imdb_id,genre,image_url,sypnosis
0,Ramayana: The Legend of Prince Rama,tt0259534,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,an anime adaptation of the hindu epic the rama...
1,The Dark Knight,tt0468569,"[Action, Crime, Drama]",http://img.omdbapi.com/?i=tt0468569&h=600&apik...,when the menace known as the joker wreaks havo...
2,The Lord of the Rings: The Return of the King,tt0167260,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,gandalf and aragorn lead the world of men agai...
3,Spider-Man: Across the Spider-Verse,tt9362722,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt9362722&h=600&apik...,miles morales catapults across the multiverse ...
4,The Message,tt0075143,"[Action, Adventure, Biography]",http://img.omdbapi.com/?i=tt0075143&h=600&apik...,the story of prophet muhammad and the delivery...


In [ ]:
from skmultilearn.model_selection import iterative_train_test_split

mlb = MultiLabelBinarizer()

def balanced_split(df, mlb, test_size=0.5):
    ind = np.expand_dims(np.arange(len(df)), axis=1)
    mlb.fit_transform(df["genre"])
    labels = mlb.transform(df["genre"])
    ind_train, _, ind_test, _ = iterative_train_test_split(
        ind, labels, test_size
    )
    return df.iloc[ind_train[:, 0]], df.iloc[ind_test[:, 0]]

In [ ]:
df_train, df_test = balanced_split(long_df, mlb, test_size=0.2)
# df_val, df_test = balanced_split(df_tmp, test_size=0.5)

In [ ]:
df_val, df_test = balanced_split(df_test, mlb, test_size=0.5)

In [ ]:
df_train.shape

(3136, 5)

In [6]:
df_train = pd.read_csv("/content/drive/MyDrive/new_stack_train.csv")
df_test = pd.read_csv('/content/drive/MyDrive/new_stack_test.csv')
df_val = pd.read_csv('/content/drive/MyDrive/new_stack_val.csv')

In [55]:
df_train = df_train.drop(columns = ["Unnamed: 0.1", "Unnamed: 0"])
df_test = df_test.drop(columns = ["Unnamed: 0.1", "Unnamed: 0"])
df_val = df_val.drop(columns = ["Unnamed: 0.1", "Unnamed: 0"])

In [7]:
df_train["genre"] = (
    df_train["genre"]
    .apply(eval)
    .apply(lambda x: [genre.strip() for genre in x])
    )

In [8]:
df_test["genre"] = (
    df_test["genre"]
    .apply(eval)
    .apply(lambda x: [genre.strip() for genre in x])
    )

In [9]:
df_val["genre"] = (
    df_val["genre"]
    .apply(eval)
    .apply(lambda x: [genre.strip() for genre in x])
    )

In [10]:
df_train = df_train.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)

In [11]:
df_train.to_csv('new_stack_train.csv')
df_test.to_csv('new_stack_test.csv')
df_val.to_csv('new_stack_val.csv')

In [ ]:
train_imgs = []

for img in tqdm(range(df_train.shape[0])):
    images_path = f"/home/jessica/code/chooj202/movie_genre_prediction/notebooks/merged/{df_train['imdb_id'][img]}.jpg"
    image = Image.open(images_path)
    image = image.resize((256, 256))
    train_imgs.append(np.array(image))
    time.sleep(1)

train_imgs = np.array(train_imgs)
np.save('/home/jessica/code/chooj202/movie_genre_prediction/notebooks/train_imgs_new', train_imgs)

100%|███████████████████████████████████████████████████████████████████████████████| 3136/3136 [55:25<00:00,  1.06s/it]


In [ ]:
np.save('/home/jessica/code/chooj202/movie_genre_prediction/notebooks/new_train_imgs', train_imgs)

In [ ]:
test_imgs = []

for img in tqdm(range(df_test.shape[0])):
    images_path = f"/home/jessica/code/chooj202/movie_genre_prediction/notebooks/merged/{df_test['imdb_id'][img]}.jpg"
    image = Image.open(images_path)
    image = image.resize((256, 256))
    test_imgs.append(np.array(image))
    time.sleep(1)

test_imgs = np.array(test_imgs)
np.save('/home/jessica/code/chooj202/movie_genre_prediction/notebooks/new_test_imgs', test_imgs)

100%|█████████████████████████████████████████████████████████████████████████████████| 402/402 [07:11<00:00,  1.07s/it]


In [ ]:
val_imgs = []

for img in tqdm(range(df_val.shape[0])):
    images_path = f"/home/jessica/code/chooj202/movie_genre_prediction/notebooks/merged/{df_val['imdb_id'][img]}.jpg"
    image = Image.open(images_path)
    image = image.resize((256, 256))
    val_imgs.append(np.array(image))
    time.sleep(1)

val_imgs = np.array(val_imgs)
np.save('/home/jessica/code/chooj202/movie_genre_prediction/notebooks/new_val_imgs', val_imgs)

100%|█████████████████████████████████████████████████████████████████████████████████| 405/405 [07:16<00:00,  1.08s/it]


In [18]:
df_train

,Unnamed: 0.1,Unnamed: 0,movie,imdb_id,genre,image_url,sypnosis,Action,Adventure,Animation,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,0,0,Ramayana: The Legend of Prince Rama,tt0259534,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,an anime adaptation of the hindu epic the rama...,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,The Dark Knight,tt0468569,"[Action, Crime, Drama]",http://img.omdbapi.com/?i=tt0468569&h=600&apik...,when the menace known as the joker wreaks havo...,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2,Spider-Man: Across the Spider-Verse,tt9362722,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt9362722&h=600&apik...,miles morales catapults across the multiverse ...,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,3,3,Inception,tt1375666,"[Action, Adventure, Sci-Fi]",http://img.omdbapi.com/?i=tt1375666&h=600&apik...,a thief who steals corporate secrets through t...,1,1,0,...,0,0,0,0,0,1,0,0,0,0
4,4,4,The Lord of the Rings: The Two Towers,tt0167261,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0167261&h=600&apik...,while frodo and sam edge closer to mordor with...,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3131,3131,3131,Bad Girls,tt0109198,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0109198&h=600&apik...,four prostitutes join together to travel the o...,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3132,3132,3132,Renegade,tt0276830,"[Adventure, Fantasy, Western]",http://img.omdbapi.com/?i=tt0276830&h=600&apik...,in after a brutal runin with an outlaw in a b...,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3133,3133,3133,Wild Wild West,tt0120891,"[Action, Comedy, Sci-Fi]",http://img.omdbapi.com/?i=tt0120891&h=600&apik...,the two best special agents in the wild west m...,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3134,3134,3134,The Ridiculous 6,tt2479478,"[Action, Adventure, Comedy]",http://img.omdbapi.com/?i=tt2479478&h=600&apik...,an outlaw who was raised by native americans d...,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_train['genre'])

#transform target variable
y_train = multilabel_binarizer.transform(df_train['genre'])
genre_names = multilabel_binarizer.classes_

# Adding
for i in range(len(genre_names)):
    df_train[f"{genre_names[i]}"] = y_train[:,i]

In [13]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_test['genre'])

#transform target variable
y_test = multilabel_binarizer.transform(df_test['genre'])
genre_names = multilabel_binarizer.classes_

# Adding
for i in range(len(genre_names)):
    df_test[f"{genre_names[i]}"] = y_test[:,i]

In [14]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_val['genre'])

#transform target variable
y_val = multilabel_binarizer.transform(df_val['genre'])
genre_names = multilabel_binarizer.classes_

# Adding
for i in range(len(genre_names)):
    df_val[f"{genre_names[i]}"] = y_val[:,i]

In [15]:
y_test.shape

(402, 21)

In [ ]:
np.save('new_y_train', y_train)
np.save('new_y_test', y_test)
np.save('new_y_val', y_val)

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/stack_train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/stack_test.csv')
df_val = pd.read_csv('/content/drive/MyDrive/stack_val.csv')

In [19]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

train_text = df_train['sypnosis'].to_list()

In [20]:
len(train_text)

3136

In [21]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

test_text = df_test['sypnosis'].to_list()

In [22]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

val_text = df_val['sypnosis'].to_list()

In [ ]:
# train_encodings_list = []
# test_encodings_list = []

# for movie in train_text:
#   train_encodings = tokenizer(str(movie), truncation=True, padding=True, max_length=128)
#   train_encodings_list.append(train_encodings['input_ids'])

# for movie in test_text:
#   test_encodings = tokenizer(str(movie), truncation=True, padding=True, max_length=128)
#   test_encodings_list.append(test_encodings['input_ids'])

# model.fit(train_encodings, tf.convert_to_tensor(y_train), epochs=3)

In [23]:
train_encodings = tokenizer(train_text, truncation = True, padding = True, max_length = 128, return_tensors = "np")
test_encodings = tokenizer(test_text, truncation = True, padding = True, max_length = 128, return_tensors = "np")
val_encodings = tokenizer(val_text, truncation = True, padding = True, max_length = 128, return_tensors = "np")

In [24]:
train_encodings['input_ids'].shape

(3136, 128)

In [ ]:
train_encodings.shape

(3136,)

In [ ]:
with open("X_test_text", "wb") as fp:   #Pickling
    pickle.dump(test_encodings_list, fp)

# >>> with open("test", "rb") as fp:   # Unpickling
# ...   b = pickle.load(fp)
# ...
# >>> b
# [1, 2, 3, 4]

In [ ]:
with open("X_train_text", "wb") as fp:   #Pickling
    pickle.dump(train_encodings_list, fp)

# >>> with open("test", "rb") as fp:   # Unpickling
# ...   b = pickle.load(fp)
# ...
# >>> b
# [1, 2, 3, 4]

In [ ]:
test_text = test_text['input_ids']

In [ ]:
train_text = train_text['input_ids']

In [25]:
train_imgs = np.load('/content/drive/MyDrive/new_train_imgs.npy')
test_imgs = np.load('/content/drive/MyDrive/new_test_imgs.npy')
val_imgs = np.load('/content/drive/MyDrive/new_val_imgs.npy')

In [ ]:
y_train = np.load('new_y_train.npy')
y_test = np.load('new_y_test.npy')
y_val = np.load('new_y_val.npy')

In [26]:
train_imgs = preprocess_input(train_imgs)
val_imgs = preprocess_input(val_imgs)
test_imgs = preprocess_input(test_imgs)

In [23]:
X_val

0      gandalf and aragorn lead the world of men agai...
1      a meek hobbit from the shire and eight compani...
2      the bride must kill her exboss and lover bill ...
3      after the rebels are overpowered by the empire...
4      teen miles morales becomes the spiderman of hi...
                             ...                        
400    to climb the corporate ladder to success a guy...
401    a high school janitor has not recovered from h...
402    as the german fascists expand their borders sc...
403    a mysterious stranger with a harmonica joins f...
404    bambino tries to teach his brother trinity how...
Name: sypnosis, Length: 405, dtype: object

In [27]:
# load the dataset and split into train/test sets
# REMEMBER TO DROP COLUMNS!!
X_train_img = train_imgs
X_train_text = train_encodings['input_ids']
y_train = y_train
X_test_img = test_imgs
X_test_text = test_encodings['input_ids']
y_test = y_test
X_val_img = val_imgs
X_val_text = val_encodings['input_ids']
y_val = y_val

In [65]:
y_train.shape

(3136, 21)

In [ ]:
train_imgs[0].shape

for img in train_imgs:
  if img.shape != (224, 224, 3):
    print("wrong dimensions")

## RESNET 50

In [28]:
def load_model():

    # $CHALLENGIFY_BEGIN

    model = ResNet50(weights="imagenet", include_top=False, input_shape=(256, 256, 3))

    # $CHALLENGIFY_END

    return model

In [29]:
def set_nontrainable_layers(model):

    # $CHALLENGIFY_BEGIN
    # Set the first layers to be untrainable
    model.trainable = False

    # $CHALLENGIFY_END

    return model

In [30]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    # $CHALLENGIFY_BEGIN
    base_model = load_model()
    base_model = set_nontrainable_layers(base_model)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(512, activation='relu')

    model = models.Sequential([
        base_model,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])
    # $CHALLENGIFY_END
    return model

## BERT Model

In [31]:
# define the BERT-based text feature extractor
def build_text_model():
 bert_model = TFBertModel.from_pretrained('bert-base-uncased')
 inputs = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='input_word_ids')
 outputs = bert_model(inputs)[1]
 text_model = tf.keras.Model(inputs=inputs, outputs=outputs)
 return text_model

## Multimodal Model

In [32]:
# define the multimodal document classification model
def build_multimodal_model(num_classes):
    model = load_model()
    model = set_nontrainable_layers(model)
    img_model = add_last_layers(model)
    text_model = build_text_model()
    img_input = tf.keras.layers.Input(shape=(256, 256, 3), name='img_input')
    text_input = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='text_input')
    img_features = img_model(img_input)
    text_features = text_model(text_input)
    concat_features = tf.keras.layers.concatenate([img_features, text_features])
    x = tf.keras.layers.Dense(512, activation='relu')(concat_features)
    x = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)
    multimodal_model = tf.keras.Model(inputs=[img_input, text_input], outputs=x)
    return multimodal_model

In [33]:
# build the multimodal model
num_classes = 21
multimodal_model = build_multimodal_model(num_classes)
multimodal_model.summary()

94765736/94765736 [==============================] - 5s 0us/step


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 img_input (InputLayer)         [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 text_input (InputLayer)        [(None, None)]       0           []                               
                                                                                                  
 sequential (Sequential)        (None, 512)          89380724    ['img_input[0][0]']              
                                                                                                  
 model (Functional)             (None, 768)          109482240   ['text_input[0][0]']       

In [34]:
# compile the model and train on the train set
multimodal_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'AUC', 'binary_accuracy', 'categorical_accuracy'])

es = EarlyStopping(monitor = 'val_accuracy',
                   mode = 'max',
                   patience = 5,
                   verbose = 1,
                   restore_best_weights = True)

history = multimodal_model.fit([(X_train_img, X_train_text)], tf.convert_to_tensor(y_train), epochs=15, batch_size=32, validation_data=([(X_val_img, X_val_text)], tf.convert_to_tensor(y_val)), callbacks = [es])

Epoch 1/15
98/98 [==============================] - 152s 1s/step - loss: 0.6592 - accuracy: 0.2640 - auc: 0.7307 - binary_accuracy: 0.8676 - categorical_accuracy: 0.2640 - val_loss: 0.3287 - val_accuracy: 0.2691 - val_auc: 0.7880 - val_binary_accuracy: 0.8800 - val_categorical_accuracy: 0.2691
Epoch 2/15
98/98 [==============================] - 96s 976ms/step - loss: 0.2225 - accuracy: 0.3807 - auc: 0.9122 - binary_accuracy: 0.9157 - categorical_accuracy: 0.3807 - val_loss: 0.3375 - val_accuracy: 0.3259 - val_auc: 0.8024 - val_binary_accuracy: 0.8794 - val_categorical_accuracy: 0.3259
Epoch 3/15
98/98 [==============================] - 95s 971ms/step - loss: 0.1199 - accuracy: 0.4142 - auc: 0.9769 - binary_accuracy: 0.9550 - categorical_accuracy: 0.4142 - val_loss: 0.3909 - val_accuracy: 0.2938 - val_auc: 0.8004 - val_binary_accuracy: 0.8772 - val_categorical_accuracy: 0.2938
Epoch 4/15
98/98 [==============================] - 95s 967ms/step - loss: 0.0591 - accuracy: 0.4359 - auc: 0.9

In [35]:
res = multimodal_model.evaluate([X_test_img, X_test_text], y_test)
res

13/13 [==============================] - 10s 428ms/step - loss: 0.3545 - accuracy: 0.3159 - auc: 0.7856 - binary_accuracy: 0.8750 - categorical_accuracy: 0.3159


[0.35447487235069275,
 0.3159204125404358,
 0.7856151461601257,
 0.8750296831130981,
 0.3159204125404358]

In [40]:
y_pred = multimodal_model.predict([X_test_img, X_test_text], verbose=1)

y_pred

13/13 [==============================] - 5s 366ms/step


array([[8.2236253e-02, 8.2993530e-02, 1.3274819e-04, ..., 3.3313847e-01,
        2.2073217e-01, 5.7364505e-02],
       [9.7507226e-01, 3.2630283e-02, 7.0102169e-04, ..., 2.3516418e-01,
        5.9916764e-02, 2.4602646e-03],
       [9.6376914e-01, 7.9833281e-01, 1.0248142e-02, ..., 4.9962099e-03,
        3.2701131e-02, 3.9017152e-02],
       ...,
       [4.4252090e-02, 1.1410884e-01, 1.4067977e-03, ..., 1.3462026e-01,
        7.4716553e-02, 3.6329467e-02],
       [5.0951988e-02, 2.3777980e-01, 2.9711396e-04, ..., 1.1669300e-01,
        2.0584992e-01, 7.3611468e-02],
       [2.5226668e-01, 4.3160986e-02, 1.2624636e-04, ..., 2.2960226e-01,
        1.0963456e-01, 8.7910652e-02]], dtype=float32)

In [43]:
multilabel_binarizer.inverse_transform(y_pred.round())[0:10]

[('Drama',),
 ('Action', 'Crime', 'Drama'),
 ('Action', 'Adventure'),
 ('Action', 'Drama', 'War'),
 ('Action', 'Crime', 'Drama'),
 ('Biography', 'Crime', 'Drama'),
 ('Action', 'Adventure'),
 ('Drama',),
 ('Action', 'Drama'),
 ('Action', 'Drama')]

In [45]:
multilabel_binarizer.inverse_transform(y_test)[0:10]

[('Action', 'Adventure', 'Biography'),
 ('Action', 'Sci-Fi'),
 ('Action', 'Drama', 'Fantasy'),
 ('Action', 'Adventure', 'Drama'),
 ('Action', 'Crime', 'Drama'),
 ('Action', 'Adventure', 'Mystery'),
 ('Action', 'Adventure', 'Fantasy'),
 ('Action', 'Drama', 'War'),
 ('Action', 'Crime', 'Drama'),
 ('Action', 'Adventure', 'Fantasy')]

In [74]:
pickle.dump(multimodal_model, open('/content/drive/MyDrive/multimodal_model.pkl','wb'))

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)
